# Structure queries with SQLAlchemy?

Test use of SQLAlchemy expression constructs instead of raw SQL for molecular queries.

In [9]:
import pandas as pd
from pandas import DataFrame

from rdkit import Chem, rdBase
from rdkit.Chem import AllChem, Draw, rdqueries, rdMolDescriptors
from rdkit.Chem.Draw import IPythonConsole

import sqlalchemy
from sqlalchemy import create_engine, Table, MetaData
from sqlalchemy.sql import select, text

In [2]:
conn = create_engine('postgresql://akokai@localhost/chmdata1')

SQLAlchemy does not recognize `rdkit:Mol` data type.

In [3]:
meta = MetaData(conn)
# meta.reflect()  # This would load metadata for all tables

# Load metadata for just the table on which we will make queries
# (Should wrap in try/except)
cpds = Table('cpds', meta, autoload=True)

/opt/akokai/anaconda/envs/cmldev/lib/python3.5/site-packages/sqlalchemy/dialects/postgresql/base.py:2517: SAWarning: Did not recognize type 'mol' of column 'molecule'
  (attype, name))


In [4]:
cpds.columns.items()

[('dtxsid', Column('dtxsid', VARCHAR(), table=<cpds>)),
 ('cid', Column('cid', TEXT(), table=<cpds>)),
 ('casrn', Column('casrn', VARCHAR(), table=<cpds>)),
 ('name', Column('name', VARCHAR(), table=<cpds>)),
 ('inchikey', Column('inchikey', VARCHAR(), table=<cpds>)),
 ('inchi', Column('inchi', VARCHAR(), table=<cpds>)),
 ('molecule', Column('molecule', NullType(), table=<cpds>))]

In [8]:
print(cpds.columns.molecule)
print(cpds.columns.molecule.key)

cpds.molecule
molecule


In [13]:
que = select([cpds]).where(cpds.c.cid == '4')
res = conn.execute(que)
ret = res.fetchall()
res.close()
print(ret)

[('DTXSID9021764', '4', '78-96-6', '1-Amino-2-propanol', 'HXKKHQJGJAFBHI-UHFFFAOYSA-N', 'InChI=1S/C3H9NO/c1-3(5)2-4/h3,5H,2,4H2,1H3', 'CC(O)CN')]


# Use custom `@>` operator: 💯

Search for phenol structures.

In [18]:
smiles = 'c1(O)ccccc1'
que = select([cpds]).where(cpds.c.molecule.op('@>')(smiles)).limit(5)
print(str(que))

SELECT cpds.dtxsid, cpds.cid, cpds.casrn, cpds.name, cpds.inchikey, cpds.inchi, cpds.molecule 
FROM cpds 
WHERE cpds.molecule @> %(molecule_1)s 
 LIMIT %(param_1)s


In [20]:
res = conn.execute(que)
print(res.rowcount, 'rows')
df = DataFrame(res.fetchall())
res.close()
df

5 rows


,0,1,2,3,4,5,6
0,DTXSID20734279,61751165,823226-19-3,1-(4-Chlorobutoxy)-4-(methanesulfonyl)benzene,HFYRLRUSFGDXMF-UHFFFAOYSA-N,"InChI=1S/C11H15ClO3S/c1-16(13,14)11-6-4-10(5-7...",CS(=O)(=O)c1ccc(OCCCCCl)cc1
1,DTXSID60735689,66743914,889362-82-7,3-tert-Butoxy-5-fluoroaniline,HMCDSAJLOQMLLW-UHFFFAOYSA-N,"InChI=1S/C10H14FNO/c1-10(2,3)13-9-5-7(11)4-8(1...",CC(C)(C)Oc1cc(N)cc(F)c1
2,DTXSID90746526,None,109738-21-8,"Oxovanadium(2+) 2,9,16,23-tetraphenoxy-20,27,2...",QLJKMZISJZORJD-UHFFFAOYSA-N,InChI=1S/C56H38N8O4.O.V/c1-5-13-33(14-6-1)65-3...,[O].[V+2].c1ccc(Oc2ccc3c4[n-]c([nH]c5[nH]c([nH...
3,DTXSID90746566,None,155773-71-0,"Nickel(2+) 1,4,8,11,15,18,22,25-octabutoxy-20,...",JGHRWXPBSMJGCS-UHFFFAOYSA-N,InChI=1S/C64H86N8O8.Ni/c1-9-17-33-73-41-25-26-...,CCCCOc1ccc(OCCCC)c2c3[n-]c([nH]c4[nH]c([nH]c5[...
4,DTXSID50746567,71311452,149820-65-5,"(8xi,9R)-6'-Methoxy-9-[(6-{[(4beta,9R)-6'-meth...",SWKRDCRSJPRVNF-RMMANSCUSA-N,InChI=1S/C56H60N6O4/c1-5-35-33-61-27-23-39(35)...,CCC1CN2CC[C@@H]1CC2[C@H](Oc1nc(-c2ccccc2)nc(O[...


# Use `qmol` type in query: ✔ (using `text`)

Search for phenyl without adjusting query (matches bezene with 1+ substituent).

In [14]:
molcol = cpds.columns.molecule
smarts = 'c1ccccc1-*'
que = select([cpds]).where(text("{0} @> '{1}'::qmol".format(molcol, smarts))).limit(5)
print(str(que))
res = conn.execute(que)
print(res.rowcount)
df = DataFrame(res.fetchall())
res.close()
df

SELECT cpds.dtxsid, cpds.cid, cpds.casrn, cpds.name, cpds.inchikey, cpds.inchi, cpds.molecule 
FROM cpds 
WHERE cpds.molecule @> 'c1ccccc1-*'::qmol 
 LIMIT %(param_1)s
5


,0,1,2,3,4,5,6
0,DTXSID20734279,61751165,823226-19-3,1-(4-Chlorobutoxy)-4-(methanesulfonyl)benzene,HFYRLRUSFGDXMF-UHFFFAOYSA-N,"InChI=1S/C11H15ClO3S/c1-16(13,14)11-6-4-10(5-7...",CS(=O)(=O)c1ccc(OCCCCCl)cc1
1,DTXSID60735689,66743914,889362-82-7,3-tert-Butoxy-5-fluoroaniline,HMCDSAJLOQMLLW-UHFFFAOYSA-N,"InChI=1S/C10H14FNO/c1-10(2,3)13-9-5-7(11)4-8(1...",CC(C)(C)Oc1cc(N)cc(F)c1
2,DTXSID70735791,66796542,51303-72-1,(2-Fluoroethanesulfonyl)benzene,KSSCPGBAPDVQEE-UHFFFAOYSA-N,"InChI=1S/C8H9FO2S/c9-6-7-12(10,11)8-4-2-1-3-5-...",O=S(=O)(CCF)c1ccccc1
3,DTXSID60735780,66788907,140449-09-8,[(Iodomethanesulfonyl)methyl]benzene,JDWWZZSCTWOKAG-UHFFFAOYSA-N,"InChI=1S/C8H9IO2S/c9-7-12(10,11)6-8-4-2-1-3-5-...",O=S(=O)(CI)Cc1ccccc1
4,DTXSID80746499,None,39001-65-5,"Zinc 2,9,16,23-tetra-tert-butyl-20,27,29,31-te...",ABEFKSDOXYFJAU-UHFFFAOYSA-N,"InChI=1S/C48H54N8.Zn/c1-45(2,3)25-13-17-29-33(...",CC(C)(C)c1ccc2c3[n-]c([nH]c4[nH]c([nH]c5[n-]c(...


# Combine RDKit query adjustments with SQLAlchemy queries

Search for phenyl and adjust query to match only for one substituent.

In [16]:
smarts = 'c1ccccc1-[*]'
qmol = Chem.AdjustQueryProperties(Chem.MolFromSmarts(smarts))
molcol = cpds.columns.molecule
smarts_adj = Chem.MolToSmarts(qmol)
que = select([cpds]).where(text("{0} @> '{1}'::qmol".format(molcol, smarts_adj))).limit(5)
print(str(que))
res = conn.execute(que)
print(res.rowcount)
df = DataFrame(res.fetchall())
res.close()
df

SELECT cpds.dtxsid, cpds.cid, cpds.casrn, cpds.name, cpds.inchikey, cpds.inchi, cpds.molecule 
FROM cpds 
WHERE cpds.molecule @> '[c&D2]1:,-[c&D2]:,-[c&D2]:,-[c&D2]:,-[c&D2]:,-[c&D3]:,-1-*'::qmol 
 LIMIT %(param_1)s
5


,0,1,2,3,4,5,6
0,DTXSID70735791,66796542,51303-72-1,(2-Fluoroethanesulfonyl)benzene,KSSCPGBAPDVQEE-UHFFFAOYSA-N,"InChI=1S/C8H9FO2S/c9-6-7-12(10,11)8-4-2-1-3-5-...",O=S(=O)(CCF)c1ccccc1
1,DTXSID60735780,66788907,140449-09-8,[(Iodomethanesulfonyl)methyl]benzene,JDWWZZSCTWOKAG-UHFFFAOYSA-N,"InChI=1S/C8H9IO2S/c9-7-12(10,11)6-8-4-2-1-3-5-...",O=S(=O)(CI)Cc1ccccc1
2,DTXSID90746526,None,109738-21-8,"Oxovanadium(2+) 2,9,16,23-tetraphenoxy-20,27,2...",QLJKMZISJZORJD-UHFFFAOYSA-N,InChI=1S/C56H38N8O4.O.V/c1-5-13-33(14-6-1)65-3...,[O].[V+2].c1ccc(Oc2ccc3c4[n-]c([nH]c5[nH]c([nH...
3,DTXSID30746505,71311253,167093-23-4,PUBCHEM_71311253,JWPNATQHLXEGSZ-UHFFFAOYSA-M,InChI=1S/C56H38N8S4.Al.ClH/c1-5-17-33(18-6-1)6...,[Al+3].[Cl-].c1ccc(Sc2cccc3c4[n-]c([nH]c5[nH]c...
4,DTXSID50746567,71311452,149820-65-5,"(8xi,9R)-6'-Methoxy-9-[(6-{[(4beta,9R)-6'-meth...",SWKRDCRSJPRVNF-RMMANSCUSA-N,InChI=1S/C56H60N6O4/c1-5-35-33-61-27-23-39(35)...,CCC1CN2CC[C@@H]1CC2[C@H](Oc1nc(-c2ccccc2)nc(O[...
